<a href="https://colab.research.google.com/github/mario-rot/AutomatedBruteForceAttack_to_FingerprintFuzzyVault/blob/main/IHLT_LABS3_readPars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 3 (Morphology) - ILTH

**Students:** Lauren Tucker & Mario Rosas

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1K4UTv_UKTe7UzcjqAi77YNodYf2vd6Kb?usp=sharing)

## Paraphrases Template

In [1]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=4c0a018049ddc0d3d37ba873bdbc2290b0c49bdb1d9280281a1a194fd22e2be9
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [2]:
import wget
URL = "https://gebakx.github.io/ihlt/sts/resources/test-gold.tgz"
response = wget.download(URL, "test_gold.tgz")

In [3]:
# importing the "tarfile" module
import tarfile

def unzip_tars(root, files, targetFolder = None):
  for fileN in files:
    file = tarfile.open(root + fileN)# open file
    if targetFolder:
      file.extractall(targetFolder) # extracting file
    else:
      file.extractall(root) # extracting file
    file.close()

In [6]:
unzip_tars('/content/', ['test_gold.tgz'], '/content/')

In [7]:
import pandas as pd

In [8]:
dt = pd.read_csv('test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)

In [9]:
dt.head()

,0,1
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi..."


In [10]:
dt['gs'] = pd.read_csv('test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

In [11]:
dt.shape

(459, 3)

In [12]:
dt.head()

,0,1,gs
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.50
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.00
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.25
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.50
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.00


# Excercise

In [13]:
from nltk.metrics import jaccard_distance
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from scipy.stats import pearsonr
import numpy as np

import string
import collections
import nltk
from nltk.corpus import gutenberg
from nltk.text import Text
nltk.download('gutenberg')
nltk.download('stopwords')

def remove_signs(wrd):
  wrd = list(wrd)
  wrd = [word for word in wrd if not any(caracter in signs for caracter in word)]
  wrd = ''.join(wrd)
  return wrd

def clean(corpus, stopwords, minwords_len, signs):
   corpus = corpus.split(' ')
   corpus = [word.lower() for word in corpus]
   corpus = [word if not any(caracter in signs for caracter in word) else remove_signs(word) for word in corpus]
   corpus = [word for word in corpus if word not in stopwords and word.isalpha()]
   corpus = [word for word in corpus if len(word) > minwords_len]
   return corpus

from nltk.corpus.reader.wordnet import VERB, NOUN, ADJ, ADV
pos_map = {'N': NOUN,
           'V':VERB,
           'J':ADJ,
           'R':ADV}

corpus = list(gutenberg.words('blake-poems.txt'))
stopwords=set(nltk.corpus.stopwords.words('english'))
signs = string.punctuation
minwords_len = 2

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
cleaned = clean(dt.loc[0][0], stopwords, minwords_len, signs)

In [15]:
pairs = nltk.pos_tag(cleaned)
pairs

[('leaders', 'NNS'),
 ('given', 'VBN'),
 ('new', 'JJ'),
 ('chance', 'NN'),
 ('let', 'VBD'),
 ('hope', 'PRP'),
 ('seize', 'VB')]

In [16]:
wnl = nltk.stem.WordNetLemmatizer()

def lemmatize(p):
    if p[1][0] in {'N','V','J','R'}:
        return wnl.lemmatize(p[0].lower(), pos=pos_map[p[1][0]])
    return p[0]

def lemmatize_sentence(sentence):
  cleaned = clean(sentence, stopwords, minwords_len, signs)
  tagged = nltk.pos_tag(cleaned)
  return[lemmatize(pair) for pair in tagged]

# [lemmatize(pair) for pair in pairs]
lemmatize_sentence(dt.loc[0][0])

['leader', 'give', 'new', 'chance', 'let', 'hope', 'seize']

In [17]:
def compute_metric(metric, elements): # Fenction to calculate Jaccard Distance
  if metric == 'jaccard':
      res = jaccard_distance(set(lemmatize_sentence(elements[0])),
                             set(lemmatize_sentence(elements[1])))
  return res

In [18]:
dt['jaccard'] = ""

In [19]:
lemmatize_sentence(dt.loc[317][0]), lemmatize_sentence(dt.loc[317][1])

(['parliament', 'adopt', 'legislative', 'resolution'],
 ['parliament', 'adopt', 'legislative', 'resolution'])

In [20]:
clean(dt.iloc[455][0], stopwords, minwords_len, signs), clean(dt.iloc[455][1], stopwords, minwords_len, signs)

(['vote', 'take', 'place', 'today'], ['vote', 'take', 'place'])

In [21]:
clean(dt.iloc[455][0], stopwords, minwords_len, signs), clean(dt.iloc[455][1], stopwords, minwords_len, signs)

(['vote', 'take', 'place', 'today'], ['vote', 'take', 'place'])

In [22]:
dt

,0,1,gs,jaccard
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.500,
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.000,
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.250,
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.500,
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.000,
...,...,...,...,...
454,It is our job to continue to support Latvia wi...,It is of our duty of continue to support the c...,5.000,
455,The vote will take place today at 5.30 p.m.,Vote will take place at 17 h 30.,4.750,
456,Neither was there a qualified majority within ...,There was no qualified majority in this Parlia...,5.000,
457,Let me remind you that our allies include ferv...,"I hold you recall that our allies, there are e...",4.000,


In [23]:
for i in range(len(dt)): # Iterating to get Jaccard distances over each column of the dataframe
  dt['jaccard'][i] = np.float64(compute_metric('jaccard',[dt.iloc[i][0],dt.iloc[i][1]]))*10

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
dt

,0,1,gs,jaccard
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.500,6.666667
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.000,0.0
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.250,7.272727
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.500,2.5
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.000,0.0
...,...,...,...,...
454,It is our job to continue to support Latvia wi...,It is of our duty of continue to support the c...,5.000,6.363636
455,The vote will take place today at 5.30 p.m.,Vote will take place at 17 h 30.,4.750,2.5
456,Neither was there a qualified majority within ...,There was no qualified majority in this Parlia...,5.000,6.666667
457,Let me remind you that our allies include ferv...,"I hold you recall that our allies, there are e...",4.000,8.0


In [25]:
pearsonr(dt['gs'], 1-dt['jaccard']) # Calculating the pearson correlation between GS results and 1-Jaccard calculated dats

(0.4810231734170826, 5.9045104154977e-28)

In [26]:
lemmatize_sentence(dt.iloc[454][0]), lemmatize_sentence(dt.iloc[454][1])

(['job',
  'continue',
  'support',
  'latvia',
  'integration',
  'russian',
  'population'],
 ['duty',
  'continue',
  'support',
  'country',
  'issue',
  'integration',
  'russian',
  'people'])

# Conclusion

Using the lemmatize function that we constructed, we obtained a higher Pearson correlation than in the prior lab. This shows that lemmatizing text samples rather than simply tokenizing them provides a more accurate measure of similarity. Compared to the gold standard, however, the performance is worse. One reason for this is that when we lemmatize, we are only looking at the word itself and analyzing whether the structure is the same. Gold standard uses more advanced techniques that analyze the meaning of each word and compare those meanings, rather than just comparing the letters. For example, in row 454, a few of the words in Sentence 1 were 'job', 'latvia', and 'population' whereas some sentences in Sentence 2 were 'duty', 'country', and 'people'. The gold standard measured a 5.0, indicating a perfect match, whereas the Jaccard distance was a 6.363636, indicating that using lemmatization these two sentences are not very similar. Since 'job' and 'duty', for example, are different words with the same meaning, lemmatization causes them to be interpreted as completely different words, but the algorithm measured by the gold standard interprets them as identical words in terms of similarity. In summary, lemmatization performs better than tokenization, but worse than the gold standard, largely due to interpreting structure instead of meaning.

